In [1]:
%load_ext autoreload
%autoreload 2
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
import numpy as np
import matplotlib.pyplot as plt
config = {
    "version": 1,
    "controllers": {
        "con1": {
            "type": "opx1",
            "analog_outputs": {
                1: {"offset": +0.0},
                2: {"offset": +0.0},
            },
        }
    },
    "elements": {
        "qe1": {
            "singleInput": {"port": ("con1", 1)},
            "intermediate_frequency": 5e6,
            "operations": {
                "playOp": "constPulse",
            },
        },
        "qe2": {
            "singleInput": {"port": ("con1", 2)},
            "intermediate_frequency": 5e6,
            "operations": {
                "playOp": "constPulse",
            },
        },
    },
    "pulses": {
        "constPulse": {
            "operation": "control",
            "length": 1000,  # in ns
            "waveforms": {"single": "const_wf"},
        },
    },
    "waveforms": {
        "const_wf": {"type": "constant", "sample": 0.2},
    },
}


# Flow control

In [3]:
with program() as prog:
    idx = declare(int)
    a = declare(int)
    with for_(idx,0,idx<10,idx+1):
        assign(a,a+1)
        save(a,"a")
    
    

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")

2021-03-30 19:34:02,719 - qm - INFO - Performing health check
2021-03-30 19:34:02,722 - qm - INFO - Health check passed
2021-03-30 19:34:02,724 - qm - WARNING - Client's version (0.90.690) does not match the server version (0.91.781).
2021-03-30 19:34:02,737 - qm - INFO - to simulate a program, use QuantumMachinesManager.simulate(..)
2021-03-30 19:34:02,745 - qm - INFO - Flags: 
2021-03-30 19:34:02,746 - qm - INFO - Simulating Qua program

The program returned the value: a=[ 1  2  3  4  5  6  7  8  9 10]


In [4]:
with program() as prog:
    a = declare(int)
    with while_(a<10):
        assign(a,a+1)
        save(a,"a")
    
    

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")

2021-03-30 19:34:44,148 - qm - INFO - Performing health check
2021-03-30 19:34:44,152 - qm - INFO - Health check passed
2021-03-30 19:34:44,155 - qm - WARNING - Client's version (0.90.690) does not match the server version (0.91.781).
2021-03-30 19:34:44,168 - qm - INFO - to simulate a program, use QuantumMachinesManager.simulate(..)
2021-03-30 19:34:44,175 - qm - INFO - Flags: 
2021-03-30 19:34:44,176 - qm - INFO - Simulating Qua program

The program returned the value: a=[ 1  2  3  4  5  6  7  8  9 10]


In [5]:
with program() as prog:
    a = declare(int)
    with while_(a<10):
        assign(a,a+1)
        with if_(a%2==0):
            save(a,"a")
            
    
    

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")

TypeError: unsupported operand type(s) for %: '_Expression' and 'int'